In [1]:
import sys
import subprocess


subprocess.run([sys.executable, "-m", "pip", "install", "pandas", "numpy", "scikit-learn"])

print("Instalación completada. Ahora importa las librerías:")
import pandas as pd
import numpy as np
print(f"Pandas {pd.__version__} y NumPy {np.__version__} instalados correctamente")

Instalación completada. Ahora importa las librerías:
Pandas 2.3.3 y NumPy 2.3.5 instalados correctamente


In [2]:
%%capture
!curl -L -o positive.csv https://raw.githubusercontent.com/Gavroshe/RuTweetCorp/master/positive.csv
!curl -L -o negative.csv https://raw.githubusercontent.com/Gavroshe/RuTweetCorp/master/negative.csv

In [10]:
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [6]:
import os
import pandas as pd

# Verifica el directorio actual
print("Directorio actual:", os.getcwd())
print("Archivos aquí:", os.listdir('.'))



positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)


negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)

df = pd.concat([positive, negative]) #This united the function -

Directorio actual: c:\Users\Usuario\Videos\Innovation Web\Project\analis_emotion\src
Archivos aquí: ['Emotions.ipynb', 'negative.csv', 'positive.csv']


In [7]:
print(df.head())

                                                text     label
0  @first_timee хоть я и школота, но поверь, у на...  positive
1  Да, все-таки он немного похож на него. Но мой ...  positive
2  RT @KatiaCheh: Ну ты идиотка) я испугалась за ...  positive
3  RT @digger2912: "Кто то в углу сидит и погибае...  positive
4  @irina_dyshkant Вот что значит страшилка :D\nН...  positive


In [8]:
df.sample(5, random_state=40)

,text,label
15931,RT @Blawar_1337: Теперь у нас с @Wake_UA появи...,positive
59532,с днём рождения зайка*))) ухх погуляем мы сего...,positive
47185,RT @Shumkova0406199: @ann_safina Вов вов вов А...,negative
42002,"Надо выдернуть звуковую дорожку из ""Доктора Ка...",positive
109035,@_hassliebe_ может все таки на этой неделе вер...,negative


We divide the data into training and test samples using the function - Разбиваем данные на обучающую и тесторовую выборки с помощью функции

In [11]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)


print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(170125,) (56709,) (170125,) (56709,)


In [12]:
y_train[:5]

90071    negative
59757    negative
35642    positive
94533    positive
90759    negative
Name: label, dtype: object

In [13]:
y_train.value_counts()

label
positive    86290
negative    83835
Name: count, dtype: int64

#Baseline: classification of unprocessed n-grams
We will try to get the transformation of sentences into a numerical vector.
Мы попробуем получить преобразование предложений в численный вектор.


In [18]:
!pip3 install nltk

In [22]:
import sys
!{sys.executable} -m pip install nltk --user

"c:\Users\Usuario\Videos\Innovation" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [21]:
import nltk
nltk.download('punkt')  # Para tokenización
nltk.download('stopwords')  # Para stopwords en español
nltk.download('wordnet')  # Para lematización

ModuleNotFoundError: No module named 'nltk'

In [23]:
import subprocess
import sys

# Instalar usando Python directamente (sin rutas con espacios)
result = subprocess.run([sys.executable, "-m", "pip", "install", "nltk"], 
                       capture_output=True, text=True)

if result.returncode == 0:
    print("✅ NLTK instalado correctamente")
else:
    print("⚠️  Intentando método alternativo...")
    # Método alternativo
    import pip
    pip.main(['install', 'nltk'])

✅ NLTK instalado correctamente


In [27]:
import nltk
from nltk import ngrams
